In [21]:
from itertools import compress
import igraph as ig

g = ig.Graph(4, directed=True)
g.add_edges([(1, 2), (2, 1), (1, 3), (0, 3), (3, 0), (2, 3)])
g.es["weight"] = [1, 10, 1, 2, 8, 9]



# there's probably a more elegant way to do this
to_delete = []
weights = g.es["weight"]
for e in compress(g.es, g.is_mutual()):
    e_opp = g.get_eid(*reversed(e.tuple))
    e = g.get_eid(*e.tuple)
    weight_dif = g.es["weight"][e] - g.es["weight"][e_opp]
    if weight_dif > 0:
        print(weight_dif)
        weights[e] = weight_dif
        to_delete.append(e_opp)

g.es["weight"] = weights
g.delete_edges(to_delete)

print(g)
print(g.es["weight"])

9
6
IGRAPH D-W- 4 4 --
+ attr: weight (e)
+ edges:
2->1 1->3 3->0 2->3
[9, 1, 6, 9]


In [31]:
from random import random
# subgraph = ig.Graph(4, directed=True)
# subgraph.add_edges([(1, 2), (2, 1), (1, 3), (0, 3), (3, 0), (2, 3)])
# subgraph.es["weight"] = [1, 10, 1, 2, 8, 9]

subgraph = ig.Graph.Erdos_Renyi(n=50, m=500, directed=True)
subgraph.es["weight"] = [random() for _ in range(500)]

print(len(subgraph.es))
print(any(subgraph.is_mutual()))

# there's probably a more elegant way to do this
to_delete = []
weights = subgraph.es["weight"]
for e in compress(subgraph.es, subgraph.is_mutual()):
    e_opp = subgraph.get_eid(*reversed(e.tuple))
    e = subgraph.get_eid(*e.tuple)
    weight_dif = weights[e] - weights[e_opp]
    if weight_dif > 0:
        weights[e] = weight_dif
        to_delete.append(e_opp)

subgraph.es["weights"] = weights
subgraph.delete_edges(to_delete)

print(len(subgraph.es))
print(any(subgraph.is_mutual()))


500
True
441
False


In [1]:
from JFRRS import JFRRS
jf = JFRRS("JFRRS.sqlite")

Connection to JFRRS.sqlite successful.


In [10]:
print(len(jf.athletes))
print(len(jf.meets))
print(jf.graph.es["meet_id"][:10])
print(len(jf.graph.clusters().giant().vs))

10102
112
[17577, 17577, 17577, 17577, 17577, 17577, 17577, 17577, 17577, 17577]
3665


In [2]:
jf.get_top_seven("MN_college_m_Carleton")

[6918559, 7907645, 6918558, 7797935, 7393503, 7393502, 6918557]

In [3]:
from pprint import pprint
pprint(jf.cxn.execute("select * from athletes where team_id == 'MN_college_m_Carleton'").fetchall())

[(6918557, 'Nakasaka, Adam', 'MN_college_m_Carleton'),
 (6918558, 'Lavey, Stephen', 'MN_college_m_Carleton'),
 (6918559, 'Santos, Ben', 'MN_college_m_Carleton'),
 (6918562, 'Shufro-Zletz, Jesse', 'MN_college_m_Carleton'),
 (7393502, 'Aoki-Sherwood, Ben', 'MN_college_m_Carleton'),
 (7393503, 'Bowman, Henry', 'MN_college_m_Carleton'),
 (7393504, 'Chao, Ben', 'MN_college_m_Carleton'),
 (7393508, 'Johnson, Parker', 'MN_college_m_Carleton'),
 (7393509, 'Jones, Elijah', 'MN_college_m_Carleton'),
 (7393510, 'Sather, Zak', 'MN_college_m_Carleton'),
 (7393511, 'Watts, Avery', 'MN_college_m_Carleton'),
 (7797931, 'Pattison, Alistair', 'MN_college_m_Carleton'),
 (7797934, 'Melville, Chris', 'MN_college_m_Carleton'),
 (7797935, 'Ash, Ethan', 'MN_college_m_Carleton'),
 (7907645, 'Hanley, Brett', 'MN_college_m_Carleton'),
 (7907646, 'Meier, Josh', 'MN_college_m_Carleton')]


In [5]:
u = jf.graph.vs.find(id=6918559) # steven
v = jf.graph.vs.find(id=7907645) # brett

jf.graph.are_connected(u, v)

False

In [2]:
lst = iter(list(range(20)))

print(next(lst))
print(next(lst))
print(next(lst))
print(next(lst))
next(lst)
next(lst)
next(lst)
print(next(lst))
print(next(lst))


0
1
2
3
7
8


In [1]:
import sqlite3

db = sqlite3.connect("JFRRS.sqlite")

In [6]:
db.execute("""
    SELECT * FROM ATHLETES WHERE TFRRS_id == 1;
""").fetchone() == None

True

In [132]:
url = "https://www.tfrrs.org/js/navbar_autocomplete.js"

with requests.get(url) as r:
    javascript = r.text

dfs = []
for table_name in ["autocomplete_teams", "autocomplete_conferences"]:
    # find meet array in retrieved javascript file
    pattern = rf"{table_name}\s*=\s*(\[(.|\n)+?\]);"
    json = re.search(pattern, javascript).group(1)
    json = json.replace("{text:", "{\"text\":").replace(",url:", ",\"url\":")

    df = pd.read_json(StringIO(json))  # load into dataframe
    
    # clean up
    df["name"] = df.text.str.replace(" (F)", "", regex=False).str.replace(" (M)", "", regex=False)

    if "teams" in table_name:
        df["gender"] = ["M" if " (M)" in text else "F" for text in df.text]

    df["id"] = (df.url
        .str.replace("/teams/", "", regex=False)
        .str.replace("/leagues/", "", regex=False)
        .str.replace("xc/", "", regex=False)
        .str.replace(".html", "", regex=False)
    )

    dfs.append(df.drop(columns=["url", "text"]).drop_duplicates())

teams, leagues = dfs

display(teams)
display(leagues)



,name,gender,id
0,Abilene Christian,F,TX_college_f_Abilene_Christian
1,Abilene Christian,M,TX_college_m_Abilene_Christian
2,Academy of Art,F,CA_college_f_Academy_of_Art
3,Academy of Art,M,CA_college_m_Academy_of_Art
4,Adams State,F,CO_college_f_Adams_State
...,...,...,...
3360,York (Pa.),M,PA_college_m_York_PA
3361,York County CC,F,ME_jcollege_f_York_County_CC
3363,York County CC,M,ME_jcollege_m_York_County_CC
3365,Youngstown St.,F,OH_college_f_Youngstown_St


,name,id
0,AARTFC,1538
1,ACC,62
2,America East,61
3,American Collegiate,1536
4,American Midwest,1477
...,...,...
174,Orange Empire,28
175,Pacific Coast,29
176,SoCal,22
177,South Coast,30


In [2]:
import dateutil
from io import StringIO
import re

import requests
import pandas as pd

# get javacript file from directathletics site
url = "https://www.directathletics.com/scripts/fuseDriver.js"
headers = {
    'authority' : 'www.directathletics.com',
    'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language' : 'en-US,en;q=0.9,la;q=0.8',
    'cache-control' : 'max-age=0',
    'cookie' : '__utma=138588838.663648652.1656164692.1656164692.1656164692.1; __utmc=138588838; __utmz=138588838.1656164692.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmv=138588838.public',
    'dnt' : '1',
    'sec-ch-ua' : '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'sec-ch-ua-mobile' : '?0',
    'sec-ch-ua-platform' : '"macOS"',
    'sec-fetch-dest' : 'document',
    'sec-fetch-mode' : 'navigate',
    'sec-fetch-site' : 'none',
    'sec-fetch-user' : '?1',
    'upgrade-insecure-requests' : '1',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
}

with requests.get(url, headers=headers) as f:
    javascript = f.text

# find meet array in retrieved javascript file
pattern = r"data\s*=\s*(\[(.|\n)+?\]);"
json = re.search(pattern, javascript).group(1).replace('\n', "")

meets = pd.read_json(StringIO(json))  # load into dataframe

# cleanup columns
meets["tfrrs"] = meets.tfrrs == "1"
meets["date"] = [dateutil.parser.parse(date) for date in meets.date_begin]

meets = (meets
    [meets.sport == "xc"]
    .drop(columns=["date_begin", "meetpro",  "sport", "outdoors", "url", "tfrrs"])
    .set_index("meet_hnd", drop=True)
    .rename_index("id")
)

meets

KeyboardInterrupt: 

In [14]:
import sqlite3

connection = sqlite3.connect("jfrrs.db")

list(connection.execute("SELECT COUNT(*) FROM results"))

[(64200,)]

In [1]:
import scrapy

In [8]:
import pandas as pd

df = pd.read_csv("prescraping/meets.csv", index_col=0)
meet_ids = reversed(df.index.to_list())
display(df)
display(meet_ids)

,venue_state,name,date
meet_hnd,,,
2052,CA,Long Beach MIddle School Cross Country Meet,2009-05-30
2080,NS,NSW Cross Country Championships,2009-06-20
2154,IN,Huntington North Cross Country Invitational,2009-08-22
2286,NC,Moore County Meet,2009-08-26
2156,IN,Purdue High School Cross Country Invitational,2009-08-29
...,...,...,...
19284,FL,NCAA Division II Cross Country Championships,2021-11-20
19300,FL,NCAA Division I Cross Country Championships,2021-11-20
19137,CA,CCCAA 2021 State Championships,2021-11-20


[19352,
 19337,
 19137,
 19300,
 19284,
 19297,
 18934,
 19289,
 18965,
 19294,
 18996,
 17754,
 19003,
 18999,
 18696,
 18697,
 18995,
 19000,
 18998,
 19295,
 19002,
 18964,
 19250,
 19024,
 19249,
 19001,
 18937,
 19292,
 19267,
 19266,
 19265,
 19264,
 18929,
 19291,
 19290,
 18936,
 19296,
 18928,
 18939,
 18935,
 19285,
 19273,
 19263,
 19197,
 19201,
 19202,
 19204,
 19200,
 19203,
 19069,
 19052,
 19261,
 19270,
 19271,
 19272,
 19040,
 19269,
 19049,
 19053,
 19039,
 19268,
 18709,
 18280,
 19066,
 19023,
 18933,
 19067,
 19050,
 19072,
 19065,
 19071,
 19070,
 19051,
 17841,
 18575,
 19195,
 19191,
 19198,
 19196,
 19199,
 19189,
 19128,
 18354,
 19046,
 18755,
 19259,
 19041,
 19258,
 19260,
 19037,
 19214,
 19257,
 19148,
 19194,
 19205,
 19255,
 19256,
 19192,
 19193,
 19190,
 19061,
 19253,
 19254,
 18520,
 19208,
 19251,
 19252,
 19183,
 19179,
 19248,
 18919,
 18736,
 18747,
 18421,
 19062,
 17988,
 19237,
 18770,
 19225,
 19019,
 18667,
 18656,
 18658,
 18661,
 18649,


In [1]:
import requests

url = "https://www.directathletics.com/scripts/fuseDriver.js"
headers = {
    "user-agent": "jfrrs"
}

with requests.get(url, headers=headers) as r:
    assert r.status_code == 200, "Request rejected by Direct Athletics"
    js = r.text.replace("\t", " ")


In [4]:
import re
import json
import pandas as pd

pattern = re.compile(r"(\[\s*(?:{.+?},?\s*)*\]);", flags=re.DOTALL)

meets_string = pattern.search(js).group(1)

,url,tfrrs,venue_state,name,meetpro,date_begin,outdoors,meet_hnd,sport
0,https://www.directathletics.com/results/track/...,0,NH,NH Weight Throw Classic,0,2009-01-02,1,10674,track
1,https://www.directathletics.com/results/swimmi...,,NH,Pinkerton vs Haverhill MA,0,2009-01-02,NaN,8708,swimming
2,https://www.directathletics.com/results/track/...,0,NH,NH Indoor Track League Meet#3 UNH Morning Session,0,2009-01-03,0,10444,track
3,https://www.directathletics.com/results/track/...,0,NH,NH Indoor Track League Meet#3 UNH Afternoon Se...,0,2009-01-03,0,10445,track
4,https://www.directathletics.com/results/track/...,0,NH,NH Indoor Track League Meet #3 Dartmouth Session,0,2009-01-03,0,10448,track
...,...,...,...,...,...,...,...,...,...
52918,//www.tfrrs.org/results/xc/20851/2022_CUNYAC_C...,1,NY,2022 CUNYAC Cross Country Championships,0,2022-10-30,NaN,20851,xc
52919,//www.tfrrs.org/results/xc/20563/NEWMAC_Cross_...,1,MA,NEWMAC Cross Country Championship,0,2022-10-30,NaN,20563,xc
52920,//www.tfrrs.org/results/xc/20757/NJCAA_Region_...,1,IA,NJCAA Region XI Cross Country Championships,0,2022-10-30,NaN,20757,xc
52921,https://www.directathletics.com/results/xc/210...,0,MA,Middlesex League XC Championship,0,2022-10-31,NaN,21071,xc


In [55]:
import json5 as json

url = "https://www.tfrrs.org/js/navbar_autocomplete.js"

with requests.get(url) as r:
    javascript = r.text

table_name = "autocomplete_teams"

# find array in javascript file
pattern = re.compile(rf"{table_name}\s*=\s*(\[.+?\]);", flags=re.DOTALL)

array_string = pattern.search(javascript).group(1)  # find array in the js file
raw_df = pd.DataFrame(json.loads(array_string))




NameError: name 'dfs' is not defined

In [3]:
from scraper.util import get_leagues_df, get_teams_df, get_meets_df

ModuleNotFoundError: No module named 'bs4'

Pipfile             docker-compose.yml  pages/              styles/
Pipfile.lock        next-env.d.ts       postcss.config.js   tailwind.config.js
README.md           next.config.js      prisma/             tsconfig.json
_legacy/            node_modules/       public/             yarn.lock
app/                package.json        scraper/
